In [1]:
import os
from const import EHR_VALID_DATA_PATH, EHR_VALID_LABEL_PATH, EHR_TRAIN_DATA_PATH, EHR_TRAIN_LABEL_PATH
import json

with open(os.path.join(EHR_VALID_DATA_PATH), "r") as f:
    valid_data = json.load(f)
with open(os.path.join(EHR_VALID_LABEL_PATH), "r") as f:
    valid_label = json.load(f)
with open(os.path.join(EHR_TRAIN_DATA_PATH), "r") as f:
    train_data = json.load(f)
with open(os.path.join(EHR_TRAIN_LABEL_PATH), "r") as f:
    train_label = json.load(f)


In [ ]:
from tqdm import tqdm
from const import SYSTEM_PROMPT
messages = []

# Train using only answerable data
for item in tqdm(train_data['data']):
    question = item['question']
    id = item['id']    
    sql = train_label.get(item['id'])  
    if sql == 'null': # Filter out unanswerable questions
        continue
    message = {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": sql}
        ]
    }
    messages.append(message)
print(len(messages)) 


  0%|          | 0/5124 [00:00<?, ?it/s]

100%|██████████| 5124/5124 [00:00<00:00, 53560.32it/s]

4674


In [3]:
print(messages[0])

{'messages': [{'role': 'system', 'content': "Given the following SQL tables and SQL assumptions you must follow, your job is to write queries given a user’s request.\n IMPORTANT: If you think you cannot predict the SQL accurately, you must answer with 'null'."}, {'role': 'user', 'content': "What's the number of patients who were laceration of intercostal blood vessels, left side, initial encounter diagnosed since 2100?"}, {'role': 'assistant', 'content': "SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT diagnoses_icd.hadm_id FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'laceration of intercostal blood vessels, left side, initial encounter' ) AND strftime('%Y',diagnoses_icd.charttime) >= '2100' )"}]}


In [4]:
import json

# Save answerable data as jsonl
with open("messages.jsonl","w",encoding="utf-8") as f:
    for entry in messages:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

### Finetuning model
Before finetuning the model, expect the cost by running `python finetunning_analysis.py`

In [3]:

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
open_ai_key = os.environ.get("OPENAI_API_KEY")
if open_ai_key is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")
client = OpenAI(api_key=open_ai_key)



In [ ]:
"""
    Uncomment to fine-tune, run this with caution (it costed about 8$)
"""

# fileobject=client.files.create(
#   file=open("messages.jsonl", "rb"),
#   purpose="fine-tune"
# )

# client.fine_tuning.jobs.create(
#   training_file=fileobject.id, 
#   model="gpt-4o-mini-2024-07-18"
# )

In [5]:
client.fine_tuning.jobs.list(limit=10)
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-RTgrxafzVr7N0fkM59Xg5ScS") 

FineTuningJob(id='ftjob-RTgrxafzVr7N0fkM59Xg5ScS', created_at=1741233639, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::B7xHlv2W', finished_at=1741235888, hyperparameters=Hyperparameters(batch_size=9, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-I29zp03jrmNMs68Tgl6V7SnV', result_files=['file-WE2gKdvEgcPZTdga8AF8HA'], seed=474235012, status='succeeded', trained_tokens=2624619, training_file='file-TkXpQXQBrF7ENuY5U2NvrR', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=9, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix=None, metadata=None)

In [19]:
# List fine-tuning jobs
jobs = client.fine_tuning.jobs.list(limit=10)
print(len(list(jobs)))  # Number of jobs

4


In [20]:
# client.fine_tuning.jobs.cancel("ftjob-YvxKrztlXNiKWVPmVhHRYHbm")